In [57]:
library(readxl)
library(tidyverse)
library(nnet)
library(lmtest)

In [58]:
# Read the Excel file
data <- read_excel("./data/exercice1.xlsx")
# Display the first few rows of the data
head(data)

ID,sexe,Region,Age,etat_civil,N_educatif,Membre_famille,Revenu_famille,Revenu_Mensuelle,Redoublant,⋯,Niveau_educ_pere,Niveau_educ_mere,Sign_ecole,Endroit_freq,Ph_famille,Ricule_Prof,Ricule_collegue,Sensation_apres_ecole,Apres_repture_ecole,Violence
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1,2,1,3,5,0,0,1,⋯,2,2,1,1,1,0,1,1,1,1
2,0,1,4,2,3,5,0,0,1,⋯,2,1,1,2,2,1,1,1,1,1
3,0,1,4,2,3,11,0,100,1,⋯,2,1,2,1,3,0,0,1,2,0
5,0,1,3,2,3,4,0,0,1,⋯,1,1,1,1,2,1,1,3,3,1
8,0,1,1,1,3,5,1,0,1,⋯,3,3,3,4,4,0,0,3,2,1
9,0,1,4,3,3,4,1,100,0,⋯,2,3,4,1,4,1,1,1,2,0


In [59]:
model1 <- multinom(Apres_repture_ecole ~ sexe + Age + Region + etat_civil
                   + N_educatif + Membre_famille + Revenu_famille
                   + Revenu_Mensuelle + Redoublant + Prob_Enseignat
                   + Aide_ecole + Rq_Comprtement + Relat_Parent
                   + Niveau_educ_pere + Niveau_educ_mere + Sign_ecole
                   + Endroit_freq + Ph_famille + Ricule_Prof
                   + Ricule_collegue, data=data)
summary(model1)

# weights:  88 (63 variable)
initial  value 124.766493 
iter  10 value 78.622797
iter  20 value 63.239268
iter  30 value 55.258599
iter  40 value 49.846960
iter  50 value 47.101855
iter  60 value 43.719936
iter  70 value 43.481027
iter  80 value 43.465618
iter  90 value 43.465149
final  value 43.465144 
converged


Warning message in sqrt(diag(vc)):
"NaNs produced"


Call:
multinom(formula = Apres_repture_ecole ~ sexe + Age + Region + 
    etat_civil + N_educatif + Membre_famille + Revenu_famille + 
    Revenu_Mensuelle + Redoublant + Prob_Enseignat + Aide_ecole + 
    Rq_Comprtement + Relat_Parent + Niveau_educ_pere + Niveau_educ_mere + 
    Sign_ecole + Endroit_freq + Ph_famille + Ricule_Prof + Ricule_collegue, 
    data = data)

Coefficients:
  (Intercept)      sexe         Age       Region  etat_civil    N_educatif
2   -7.489772 245.87263  -0.4376716   -2.4084838    4.901879 -5.771419e-01
3  -10.469941  47.08085   0.7066992    0.6411367    5.941687  8.525498e-03
4 -611.468711  82.99164 208.3453744 -225.7999524 -128.520080 -1.605677e+02
  Membre_famille Revenu_famille Revenu_Mensuelle   Redoublant Prob_Enseignat
2      0.2134464      0.4469855       0.00356250    0.6462154      1.3745384
3      0.1245541      0.7613484       0.00124591   -0.2674273     -0.3408284
4     23.4909023    -32.9186956       0.42818081 -187.7576202    -38.2412999
  Aide